In [1]:
import tensorflow as tf
import numpy as np

In [2]:
"""
第一层为卷积层, 卷积层_1的参数  
input:  filter: 
        size: 5*5;  number: 6  channel:  1  
output: feature map: 
        size 28*28, number 6; channel :1
"""
CONV1_FILTER_SIZE = 5
CONV1_FILTER_CHANNEL = 1
CONV1_FILTER_NUM = 6

In [3]:
"""
第三层为卷积层, 卷积层_3的参数
input: filter:
       size 5 * 5   number 16 channel: 1
output: feature map
       size 10 * 10 number 16  channel: 1 
       
"""
CONV3_FILTER_SIZE = 5
CONV3_FILTER_CHANNEL = 1
CONV3_FILTER_NUM = 16

In [4]:
"""
第五层是一个卷积层, 
input: filter:
        size 5 *5    number 120   channel: 1
output: feature map
        size 1    number 120  channel: 1
"""
CONV5_FILTER_SIZE = 5
CONV5_FILTER_CHANNEL = 1
CONV5_FILTER_NUM = 120

# LeNet_inference

In [5]:
def LeNet_inference(inTensor):
    
    """conv 1.  第一层为卷积层,有6个filter
    input size: 32 * 32 * 1
    output size: 28 * 28 * 6
    """
    conv1_filter = tf.Variable(tf.truncated_normal([CONV1_FILTER_SIZE,CONV1_FILTER_SIZE,CONV1_FILTER_CHANNEL, CONV1_FILTER_NUM],dtype=tf.float32))
    conv1 = tf.nn.conv2d(inTensor, conv1_filter, strides=[1, 1, 1, 1], padding='VALID')
    conv1_bias = tf.Variable(tf.truncated_normal([CONV1_FILTER_NUM], dtype=tf.float32))
    relu_1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_bias))
    
    """pooling 2.  
     第2层为pooling层,与原始LetNet不同.原始的LeNet的pooling层有两个可训练参数
     input size: 28 * 28 * 6
     output size: 14 * 14 * 6
    """
    pooling2 = tf.nn.avg_pool(relu_1, ksize=[1,2,2,1], strides=[1, 2, 2 ,1], padding="VALID")
    relu_2 = tf.nn.relu(pooling2)
    
    
    """ conv 3  
    第3层为卷积层,共16个filter. 与原始LeNet不同,原始的LeNet是部分连接
    这里为了方便是,整体连接,详细情况见论文希捷
    input size 14 * 14 * 6
    out putsize 10 * 10 *16
    """
    conv3_filter = tf.Variable(tf.truncated_normal([5, 5, 6, 16]))
    conv3_bias = tf.Variable(tf.truncated_normal([16]))
    conv3 = tf.nn.conv2d(relu_2, conv3_filter, strides=[1,1,1,1],padding='VALID')
    relu_3 =tf.nn.relu(tf.nn.bias_add(conv3, conv3_bias))
    
    
    """pooling 4  
    第4层为pooling层, 与pooling 2 相同
    input size:  10 * 10 * 16
    output size: 5 * 5 * 16
    """
    pooling4 = tf.nn.avg_pool(relu_3, ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    relu_4 = tf.nn.relu(pooling4)
    
    """conv 5
    第5层为conv层, 这一层与原始的LeNet-5是一致的
    input size: 5 * 5 * 16
    output size: 1 * 1 * 120
    """
    conv5_filter = tf.Variable(tf.truncated_normal([5, 5, 16, 120]))
    conv5_bias = tf.Variable(tf.truncated_normal([120]))
    conv5 = tf.nn.conv2d(relu_4, conv5_filter, strides=[1,1,1,1],padding="VALID")
    relu_5 = tf.nn.relu(tf.nn.bias_add(conv5, conv5_bias))
    
    """fc 6
    第6层为fc层,与原始不太一致,因为我只在mnist上进性试验
    input size: 1 * 1 * 120
    output size: 1* 1* 10
    """
    fc_weights = tf.Variable(tf.truncated_normal([120,10]))
    fc_bias = tf.Variable(tf.truncated_normal([10]))
    relu_5 = tf.reshape(relu_5, [1, 120])
    relu_6 = tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(relu_5, fc_weights),fc_bias))
    
    
    """最后的输出单元
    softmax分类器
    """
    logits = tf.nn.softmax(relu_6)
    #result = tf.one_hot(tf.argmax(tf.transpose(logits)), 10)
    return logits
    
    
    
    

#  进行测试

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./input_data', one_hot=True, validation_size=100)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./input_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./input_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./input_data\t10k-images-idx3-ubyte.gz
Extracting ./input_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
import matplotlib.pyplot as plt
ind = 10;
test = mnist.train.images[ind, :]
label = mnist.train.labels[ind, :]
shaped = test.reshape((28, 28))
padded = np.pad(shaped, ((2,2),(2,2)), "constant", constant_values=0)

In [25]:
im = np.reshape(padded, (1, 32, 32, 1))
result = LeNet_inference(im)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=result )
loss = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
a = LeNet_inference(im)

运行loss function

In [37]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(result))
    for i in range(100):
        sess.run(train_op)
 
    

[[ 0.04539683  0.12340137  0.04539683  0.12340137  0.04539683  0.12340137
   0.12340137  0.12340137  0.12340137  0.12340137]]
